In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Set the group number
group_number = 1  # Change this to the desired group number

# Load the CSV files from the specified group
sdl_class_rdoc_path = f'0.class_document/{group_number}/sdl_class_rdoc.csv'
test_path = f'0.class_document/{group_number}/test_p_c.csv'

sdl_class_rdoc_csv = pd.read_csv(sdl_class_rdoc_path, low_memory=False)
test_csv = pd.read_csv(test_path, low_memory=False)

update_count = 0
duplicate_count = 0
non_duplicate_count = 0

# Assign c_thing, c_property to p_thing, p_property and set p_MDM to True if conditions are met
for index, row in test_csv.iterrows():
    if not row['p_correct'] and row['ctp_correct']:
        update_count += 1  # Increment the counter

        # Check for duplicates within the same ships_idx
        same_idx_rows = test_csv[(test_csv['ships_idx'] == row['ships_idx']) &
                                 (test_csv['p_thing'] == row['c_thing']) &
                                 (test_csv['p_property'] == row['c_property'])]

        if len(same_idx_rows) > 0:
            duplicate_count += 1
        else:
            non_duplicate_count += 1

# Print the results
print(f"Total updates where p_correct is False and ctp_correct is True: {update_count}")
print(f"Number of rows with duplicates in the same ships_idx: {duplicate_count}")
print(f"Number of rows without duplicates in the same ships_idx: {non_duplicate_count}")



KeyError: 'p_correct'

In [ ]:
update_count = 0

# Initialize a list to hold rows that meet the conditions
refine_rows = []

# Assign c_thing, c_property to p_thing, p_property and set p_MDM to True if conditions are met
for index, row in test_csv.iterrows():
    if (not row['p_MDM'] and row['c_score'] >= 0.9 and 
        (row['p_thing'] != row['c_thing'] or row['p_property'] != row['c_property'])):
        test_csv.at[index, 'p_thing'] = row['c_thing']
        test_csv.at[index, 'p_property'] = row['c_property']
        test_csv.at[index, 'p_MDM'] = True
        update_count += 1  # Increment the counter
        refine_rows.append(row)  # Add the row to the refine list

# Convert the list of refine rows into a DataFrame
refine_df = pd.DataFrame(refine_rows)

# Save the refine DataFrame to a CSV file
refine_output_path = f'refine.csv'
refine_df.to_csv(refine_output_path, index=False, encoding='utf-8-sig')

# Print the number of updates made
print(f"Number of updates made: {update_count}")

# Save the updated test CSV
output_file_path = f'0.class_document/{group_number}/test_p_c_r.csv'
test_csv.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
print(f"Updated test CSV saved to {output_file_path}")
print(f"Refine CSV saved to {refine_output_path}")


Number of updates made: 45
Updated test CSV saved to 0.class_document/1/test_p_c_r.csv
Refine CSV saved to refine.csv
